# 上級編1

In [56]:
import numpy as np
import pandas as pd
from collections import Counter

In [69]:
# 生成多項式から生成行列を作る場合
# g = np.array([0, 0, 0, 1, 1, 0, 1])
g = np.array([0, 0, 1, 1, 1, 0, 1])
k = np.argmax(g) + 1
n = row.shape[0]
G = np.stack([g] * k, 0)
for i, row in enumerate(G):
    if i >= 1:
        G[i] = np.roll(G[i - 1], -1)
G = np.matrix(G, dtype="u1")
print(G)

[[0 0 1 1 1 0 1]
 [0 1 1 1 0 1 0]
 [1 1 1 0 1 0 0]]


In [70]:
# 課題の生成行列
# G = np.matrix([
#     [1, 0, 1, 0, 1, 1],
#     [1, 1, 1, 1, 0, 1],
#     [0, 1, 1, 0, 0, 1]
# ], dtype="u1")

# 崩したハミング符号
# 1ビット以下の誤りのみ検出可能
# 完全符号
# G = np.matrix([
#     [1, 1, 0, 0, 1, 0, 0],
#     [1, 0, 1, 0, 0, 1, 0],
#     [1, 0, 0, 1, 0, 0, 1],
#     [1, 1, 1, 0, 0, 0, 1]
# ], dtype="u1")

# G = np.matrix([
#     [0, 1, 1, 1, 1, 0, 0, 0],
#     [1, 1, 1, 0, 0, 1, 0, 0],
#     [1, 1, 0, 1, 0, 0, 1, 0],
#     [1, 0, 1, 1, 0, 0, 0, 1]
# ], dtype="u1")

k, n = G.shape
print(G)
k, n

[[0 0 1 1 1 0 1]
 [0 1 1 1 0 1 0]
 [1 1 1 0 1 0 0]]


(3, 7)

## 掃き出し法で組織符号の生成行列に変換

In [71]:
k = G.shape[0]

A = np.matrix(np.identity(k), dtype="u1")
G_src = G.copy()[:,:k]
I_k = np.identity(k)
# print(G_src)
for i in range(k):
#     print(f"掃き出し{i + 1}回目")
    col_i = np.array(G_src)[:,i]
    # 対角成分が0
    # 他の行と入れ替え
    if col_i[i] == 0:
        # 入れ替え対象の行番号
        j = np.where(col_i == 1)[0][-1]
        A_i = I_k.copy()
        A_i[i] = I_k[j].copy()
        A_i[j] = I_k[i].copy()
        A_i = np.matrix(A_i, dtype="u1")
        A = A_i * A % 2
        G_src = A_i * G_src % 2
#         print(G_src)
        col_i = np.array(G_src)[:,i]
    A_i =I_k.copy()
    A_i[:,i] = col_i
    A_i = np.matrix(A_i, dtype="u1")
    A = A_i * A % 2
    # print(f"A_i=\n{A_i}")
    # print(f"A=\n{A}")
    G_src = A_i * G_src % 2
#     print(G_src)

Tissue_G = A * G % 2
print(f"A=\n{A}")
print(f"G=\n{Tissue_G}")

A=
[[0 1 1]
 [1 1 0]
 [1 0 0]]
G=
[[1 0 0 1 1 1 0]
 [0 1 0 0 1 1 1]
 [0 0 1 1 1 0 1]]


## 標準アレイ

In [36]:
all_src_ex0 = np.matrix([[int(j) for j in format(i, f"0{k}b")] for i in range(1, 2 ** k)], dtype="u1")
all_code_ex0 = all_src_ex0 * Tissue_G % 2
all_code_ex0_str = ["".join(str(i) for i in l) for l in all_code_ex0.tolist()]
all_code_ex0_str

['0001110',
 '0010101',
 '0011011',
 '0100011',
 '0101101',
 '0110110',
 '0111000',
 '1000111',
 '1001001',
 '1010010',
 '1011100',
 '1100100',
 '1101010',
 '1110001',
 '1111111']

In [37]:
# 1ビットエラー全パターン?
# all_error_str = [format(2 ** i, f"0{n}b") for i in range(n)]
all_one_bit_error = [format(2 ** i, f"0{n}b") for i in range(n)]
custom_error_str = []
# print(all_one_bit_error)
# 適当に追加
# all_error_str += ["000011"]
# 課題の (8, 4) 符号用
# custom_error_str = ["00000011", "00000101", "00001001", "00010001", "00100001", "01000001", "10000001"]
# 起こりやすい誤りを勝手に定義
# all_error_str = ["000001", "000010", "000100", "001000", "010000", "000011", "000101"]
all_error_str = all_one_bit_error + custom_error_str
all_error = np.matrix([[int(j) for j in moji] for moji in all_error_str])
all_error_str

['0000001', '0000010', '0000100', '0001000', '0010000', '0100000', '1000000']

In [38]:
std_arr_matrix = [[np.matrix((e + w) % 2) for w in np.array(all_code_ex0)] for e in np.array(all_error)]
# std_arr_values = [["".join(str(k) for k in (e + w) % 2) for w in np.array(all_code_ex0)] for e in np.array(all_error)]
std_arr_values = [["".join(str(k) for k in j.tolist()[0]) for j in i] for i in std_arr_matrix]
# np.array(std_arr_values)
std_arr_df = pd.DataFrame(std_arr_values, columns=all_code_ex0_str, index=all_error_str)
std_arr_df

,0001110,0010101,0011011,0100011,0101101,0110110,0111000,1000111,1001001,1010010,1011100,1100100,1101010,1110001,1111111
0000001,0001111,0010100,0011010,0100010,0101100,0110111,0111001,1000110,1001000,1010011,1011101,1100101,1101011,1110000,1111110
0000010,0001100,0010111,0011001,0100001,0101111,0110100,0111010,1000101,1001011,1010000,1011110,1100110,1101000,1110011,1111101
0000100,0001010,0010001,0011111,0100111,0101001,0110010,0111100,1000011,1001101,1010110,1011000,1100000,1101110,1110101,1111011
0001000,0000110,0011101,0010011,0101011,0100101,0111110,0110000,1001111,1000001,1011010,1010100,1101100,1100010,1111001,1110111
0010000,0011110,0000101,0001011,0110011,0111101,0100110,0101000,1010111,1011001,1000010,1001100,1110100,1111010,1100001,1101111
0100000,0101110,0110101,0111011,0000011,0001101,0010110,0011000,1100111,1101001,1110010,1111100,1000100,1001010,1010001,1011111
1000000,1001110,1010101,1011011,1100011,1101101,1110110,1111000,0000111,0001001,0010010,0011100,0100100,0101010,0110001,0111111


In [39]:
# 重複がないか確認
len(set(std_arr_df.values.ravel()))

105

## 各受信後のシンドローム計算

In [42]:
# パリティ生成部分
P = Tissue_G[:, k:]
# print(P)
# 検査行列
H = np.concatenate([P.T, np.identity(n - k, dtype=np.uint8)], 1)
H

matrix([[1, 0, 1, 1, 1, 0, 0],
        [1, 1, 0, 1, 0, 1, 0],
        [1, 1, 1, 0, 0, 0, 1]], dtype=uint8)

In [43]:
syndrome_values = [["".join(str(k) for k in (j * H.T % 2).tolist()[0]) for j in i] for i in std_arr_matrix]
syndrome_df = pd.DataFrame(syndrome_values, columns=all_code_ex0_str, index=all_error_str)
syndrome_df

,0001110,0010101,0011011,0100011,0101101,0110110,0111000,1000111,1001001,1010010,1011100,1100100,1101010,1110001,1111111
0000001,001,001,001,001,001,001,001,001,001,001,001,001,001,001,001
0000010,010,010,010,010,010,010,010,010,010,010,010,010,010,010,010
0000100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100
0001000,110,110,110,110,110,110,110,110,110,110,110,110,110,110,110
0010000,101,101,101,101,101,101,101,101,101,101,101,101,101,101,101
0100000,011,011,011,011,011,011,011,011,011,011,011,011,011,011,011
1000000,111,111,111,111,111,111,111,111,111,111,111,111,111,111,111


## 最小ハミング距離

In [44]:
# G = np.matrix([
#     [1, 0, 0, 0, 0, 1, 1, 1],
#     [0, 1, 0, 0, 1, 1, 1, 0],
#     [0, 0, 1, 0, 1, 1, 0, 1],
#     [0, 0, 0, 1, 1, 0, 1, 1]
# ])

# k, n = G.shape
print(G)
print(k, n)

all_src_ex0 = [np.matrix([int(j) for j in format(i, f"0{k}b")], dtype=np.uint8) for i in range(1, 1 << k)]
all_code_ex0 = [i * G % 2 for i in all_src_ex0]
all_code_ex0_hamming_dist = [i.sum() for i in all_code_ex0]
hamming_dist_min = min(all_code_ex0_hamming_dist)
print(f"最小ハミング距離: {hamming_dist_min}")
# all_code_ex0

[[1 1 0 0 1 0 0]
 [1 0 1 0 0 1 0]
 [1 0 0 1 0 0 1]
 [1 1 1 0 0 0 1]]
4 7
最小ハミング距離: 3


In [46]:
A_i_dic = Counter(all_code_ex0_hamming_dist)
print(A_i_dic)
p = 0.01
P_u_E = 0
for i, A_i in A_i_dic.items():
    P_u_E += A_i * p ** i * (1 - p) ** (n - i)
P_u_E

Counter({4: 7, 3: 7, 7: 1})


6.7920930100000004e-06